In [1]:
import pandas as pd
import pymysql as mc 
import os
import sys
import time
import numpy as np

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [60]:
cnx = mc.connect(user='akashgoyal',password='Metrostars3',
                 host='stromberg.cs.uchicago.edu',db='mlb_practicum',port=3306)

In [3]:
query = """SELECT p1.gameID,p1.playerID,firstName,lastName,batPosition,
            teamAbbreviation AS player_mlb_team,100_avg,300_avg,500_avg, 1000_avg,
            FROM Player p1
            INNER JOIN Batter_Run_Expectancy b1 ON
            (p1.gameID = b1.gameID AND p1.playerID = b1.playerID)"""
df = pd.read_sql_query(query,cnx)

In [17]:
starters = df[(df['batPosition'].notnull()) & df['batPosition']!=0]

In [18]:
pivot_tab = pd.pivot_table(starters,values=['100_avg','300_avg','500_avg','1000_avg'],
               #columns=['100_avg'],#'300_avg','500_avg','1000_avg'],
               index=['gameID','player_mlb_team'], aggfunc=np.average)

In [21]:
from sqlalchemy import create_engine

In [22]:
pw = '******'
user = 'akashgoyal'
host = 'stromberg.cs.uchicago.edu'
db_name = 'mlb_practicum'

engine = create_engine('mysql+pymysql://'+user+':'+pw+'@'+host+':3306/'+db_name, echo=False)
#df.to_sql(name='Fake_Data',con=engine,if_exists='append',index=False)

In [24]:
pivot_tab.to_sql(name='Team_wrc_stats',con=engine,index=True)

InternalError: (pymysql.err.InternalError) (1170, "BLOB/TEXT column 'player_mlb_team' used in key specification without a key length") [SQL: 'CREATE INDEX `ix_Team_wrc_stats_player_mlb_team` ON `Team_wrc_stats` (player_mlb_team)'] (Background on this error at: http://sqlalche.me/e/2j85)

In [27]:
pivot_tab.to_csv('team_wrc_stats.csv')

In [61]:
query2 = "SELECT gameID, pitchID,x,y FROM Pitch2"
df2 = pd.read_sql_query(query2,cnx)

In [52]:
def reclassify_location(df):
    if df['y']>153.7:
        if df['x']<94.9:
            return 1
        elif df['x']<117.2:
            return 2
        else:
            return 3
    elif df['y']>139.3:
        if df['x']<94.9:
            return 4
        elif df['x']<117.2:
            return 5
        else:
            return 6
    else:
        if df['x']<94.9:
            return 7
        elif df['x']<117.2:
            return 8
        else:
            return 9

In [62]:
df2['zone'] = df2.apply(reclassify_location,axis=1)

In [63]:
df2.head()

,gameID,pitchID,x,y,zone
0,ANA201004050,3,109.87,145.92,5
1,ANA201004050,4,119.31,140.74,6
2,ANA201004050,5,92.70,202.91,1
3,ANA201004050,6,122.75,144.20,6
4,ANA201004050,7,101.29,166.65,2


In [64]:
pw = 'Metrostars3'
user = 'akashgoyal'
host = 'stromberg.cs.uchicago.edu'
db_name = 'mlb_practicum'

engine = create_engine('mysql+pymysql://'+user+':'+pw+'@'+host+':3306/'+db_name, echo=False)
df2.to_sql(name='Pitch_Coordinates',con=engine,if_exists='append',index=False)